In [ ]:
import plotly.graph_objects as go
import ipywidgets as widgets
from scipy.spatial.transform import Rotation
import numpy as np
import magpylib as magpy
import plotly.figure_factory as ff

debug_view = widgets.Output(layout={'border': '1px solid black'})

# Define magnet box

In [ ]:
@debug_view.capture(clear_output=True)
def _define_magnet_box(pos = (0,0,0), dim = (10,10,10), angle=0, axis=None):
    box = dict(
            i = np.array([7, 0, 0, 0, 4, 4, 2, 6, 4, 0, 3, 7]),
            j = np.array([3, 4, 1, 2, 5, 6, 5, 5, 0, 1, 2, 2]),
            k = np.array([0, 7, 2, 3, 6, 7, 1, 2, 5, 5, 7, 6]),
            x = np.array([-1, -1, 1, 1, -1, -1, 1, 1])*0.5*dim[0]+pos[0],
            y = np.array([-1, 1, 1, -1, -1, 1, 1, -1])*0.5*dim[1]+pos[1],
            z = np.array([-1, -1, -1, -1, 1, 1, 1, 1])*0.5*dim[2]+pos[2]
    )
    if angle!=0:
        rotation = Rotation.from_rotvec(np.deg2rad(angle)*np.array(axis))
        c = np.array([box['x'],box['y'], box['z']])
        Mm = (np.max(c,axis=1)+ np.min(c,axis=1))/2          
        box_rotated = rotation.apply(c.T-Mm) + Mm
        box['x'],box['y'], box['z'] = box_rotated.T
    return box.copy()


# Show streamlines

In [ ]:
@debug_view.capture(clear_output=True)
def show_streamlines(change=None):
        sk = streamlines_options
        test=[]
        for k,v in sk.items():
            test.append(v['checkbox'].value and v['density'].value>0.5)
        if sum(test)>0:
            continuous_update_checkbox.value = False
        
        
        try:

            sr = 100
            N=2
            if sk['xy']['checkbox'].value == True:
                pos = sk['xy']['position'].value
                density = sk['xy']['density'].value
                xs= np.linspace(-sr,sr,int(N*density))
                ys= np.linspace(-sr,sr,int(N*density))
                Bs = np.array([[pmc.getB([x,y,pos]) for x in xs] for y in ys])
                U,V = Bs[:,:,0], Bs[:,:,1]
                streamline = ff.create_streamline(x=xs, y=ys , u=U, v=V, density = 0.5*density, arrow_scale=sr*0.05)
                sl = streamline.data[0]
                t = figmag.data[1]
                with figmag.batch_update():
                    t.visible = True
                    t.x=sl.x ; t.y=sl.y ;  t.z=np.ones(len(sl.x))*pos
            else:
                figmag.data[1].visible = False

            if sk['xz']['checkbox'].value == True:
                pos = sk['xz']['position'].value
                density = sk['xz']['density'].value
                xs= np.linspace(-sr,sr,int(N*density))
                zs= np.linspace(-sr,sr,int(N*density))
                Bs = np.array([[pmc.getB([x,pos,z]) for x in xs] for z in zs])
                U,V = Bs[:,:,0], Bs[:,:,2]
                streamline = ff.create_streamline(x=xs, y=zs , u=U, v=V, density = 0.5*density, arrow_scale=sr*0.05)
                sl = streamline.data[0]
                t = figmag.data[2]
                with figmag.batch_update():
                    t.visible = True
                    t.x=sl.x ; t.y=np.ones(len(sl.x))*pos ;  t.z=sl.y
            else:
                figmag.data[2].visible = False

            if sk['yz']['checkbox'].value == True:
                pos = sk['yz']['position'].value
                density = sk['yz']['density'].value
                ys= np.linspace(-sr,sr,int(N*density))
                zs= np.linspace(-sr,sr,int(N*density))
                Bs = np.array([[pmc.getB([pos,y,z]) for y in ys] for z in zs])
                U,V = Bs[:,:,1], Bs[:,:,2]
                streamline = ff.create_streamline(x=ys, y=zs , u=U, v=V, density = 0.5*density, arrow_scale=sr*0.05)
                sl = streamline.data[0]
                t = figmag.data[3]
                with figmag.batch_update():
                    t.visible = True
                    t.x=np.ones(len(sl.x))*pos ; t.y=sl.x ;  t.z=sl.y
            else:
                figmag.data[3].visible = False
            
        except:
            pass
    

# Update magnet

In [ ]:
@debug_view.capture(clear_output=True)
def update_magnet(mag_id, angle, axis_x, axis_y, axis_z, xpos, ypos,zpos, Lx, Ly, Lz, Mx, My, Mz):
    global magnets
    axis = (axis_x, axis_y, axis_z)
    mp = magnets[mag_id]
    pos,dim = mp['properties'].values() 
    if angle!=0:
        mp['magpy_magnet'].setOrientation(angle, [axis_x,axis_y,axis_z])
    pos = mp['magpy_magnet'].position = np.array([xpos, ypos, zpos])
    dim = mp['magpy_magnet'].dimension = np.array([Lx,Ly,Lz])
    mag_val = mp['magpy_magnet'].magnetization = np.array([Mx,My,Mz])
    tm = mp['trace']
    
    show_streamlines()
        
    with figmag.batch_update():
        i, j, k, tm.x, tm.y, tm.z = _define_magnet_box(pos, dim, angle, axis).values()
        tm.intensity = getIntensity(tm, rotation.apply(mag_val), pos)
        
        
@debug_view.capture(clear_output=True)
def on_continuous_update_change(change):
    global wmag
    for child in wmag.children[1:-1]:
        child.continuous_update = change.owner.value
        
        
@debug_view.capture(clear_output=True)
def update_magnet_title(change):
    magnets_list_widget.set_title(magnets_list_widget.selected_index, f"{change.owner.id} {change.new}")


# Delete magnet

In [ ]:
@debug_view.capture(clear_output=True)
def on_delete_magnet_button_clicked(b):
    global magnets
    pmc.removeSource(magnets[b.id]['magpy_magnet'])
    magnets.pop(b.id, magnets)
    figmag.data = [figmag.data[0],figmag.data[1]] + [v['trace'] for k,v in magnets.items()]
    magnets_list_widget.children = [v['widget'] for k,v in magnets.items()]
    for i,n in enumerate([v['name'].value for k,v in magnets.items()]):
        magnets_list_widget.set_title(i,f"{b.id} {n}")
        
def delete_all_magnets():
    figmag.data = figmag.data[:4]
    magnets =dict()
    for m in magnets.values():
        pmc.removeSource(m['magpy_magnet'])
    magnets_list_widget.children=[]

# Add magnet

In [ ]:
def getIntensity(cube, mag, pos):
    x,y,z = np.array(mag) / np.linalg.norm(mag)
    return ((cube.x-pos[0])*x + (cube.y-pos[1])*y + (cube.z-pos[2])*z) / np.sqrt((cube.x-pos[0])**2 + (cube.y-pos[1])**2 + (cube.z-pos[2])**2)


@debug_view.capture(clear_output=True)
def add_magnet(mag=(0,0,10),dim=(10,10,10), pos=(0,0,0), name=None):
    global wmag
    for i in range(1,100):
        mag_id = f'{i:02d} 🧲'
        if not mag_id in magnets.keys():
            break
            
    delete_magnet_button = widgets.Button(description='delete', icon='trash', button_style='danger',
                                         layout=dict(width='auto'))
    delete_magnet_button.on_click(on_delete_magnet_button_clicked)
    delete_magnet_button.id = mag_id
    magname_widget = widgets.Text(description='name', value = '' if name == None else name)
    magname_widget.id = mag_id
    magname_widget.observe(update_magnet_title, names = 'value')
    mag_props  = {'pos' : pos,  'dim' : dim}
    cst = 0.1 #color scale threshold
    figmag.add_mesh3d(opacity = 1, **_define_magnet_box(pos, dim), name=mag_id, 
                      colorscale=[[0, 'turquoise'], [0.5-cst, 'turquoise'],[0.5+cst, 'magenta'], [1, 'magenta']], 
                      showscale=False)
    figmag.data[-1].intensity = getIntensity(figmag.data[-1],mag, pos)
    magpy_magnet = magpy.source.magnet.Box(mag=mag, pos=pos ,dim=dim)
    pmc.addSources(magpy_magnet)
    orientation_widgets = dict(angle = widgets.FloatSlider(description='angle [deg]', min=-180, max=180, value=0, 
                                                           style=dict(handle_color='blue')),
                               axis_x= widgets.FloatSlider(description='x', min=-1, max=1, step=0.1, value=0),
                               axis_y= widgets.FloatSlider(description='y', min=-1, max=1, step=0.1, value=0),
                               axis_z= widgets.FloatSlider(description='z', min=-1, max=1, step=0.1, value=1)
                              )
    position_widgets = dict(xpos = widgets.FloatSlider(description='x [mm]', min=pos[0]-5, max=pos[0]+5, step=0.1, value=pos[0]),
                            ypos = widgets.FloatSlider(description='y [mm]', min=pos[1]-5, max=pos[1]+5, step=0.1, value=pos[1]),
                            zpos = widgets.FloatSlider(description='z [mm]', min=pos[2]-5, max=pos[2]+5, step=0.1, value=pos[2])
                           )
    dimensions_widgets = dict(Lx = widgets.FloatSlider(description='x [mm]', min=0, max=dim[0]*5, step=0.1, value=dim[0]),
                            Ly = widgets.FloatSlider(description='y [mm]', min=0, max=dim[1]*5, step=0.1, value=dim[1]),
                            Lz = widgets.FloatSlider(description='z [mm]', min=0, max=dim[2]*5, step=0.1, value=dim[2])
                           )
    magnetization_widgets = dict(Mx = widgets.FloatSlider(description='x [mT]', min=-abs(mag[0])*2-1, max=abs(mag[0])*2+1, step=0.1, value=mag[0]),
                            My = widgets.FloatSlider(description='y [mT]', min=-abs(mag[1])*2-1, max=abs(mag[1])*2+1, step=0.1, value=mag[1]),
                            Mz = widgets.FloatSlider(description='z [mT]', min=-abs(mag[2])*2-1, max=abs(mag[2])*2+1, step=0.1, value=mag[2])
                                )
    all_widgets = dict(**orientation_widgets, **position_widgets, **dimensions_widgets, **magnetization_widgets)
    
    for w in all_widgets.values():
        w.style.description_width='auto'
        w.layout.width='auto'
    wmag = widgets.interactive(update_magnet, mag_id= widgets.Label(mag_id), **all_widgets)
    
    tabs = widgets.Tab([widgets.VBox(list(position_widgets.values()) + [continuous_update_checkbox]),
                        widgets.VBox(list(dimensions_widgets.values()) + [continuous_update_checkbox]),
                        widgets.VBox(list(orientation_widgets.values()) + [continuous_update_checkbox]), 
                        widgets.VBox(list(magnetization_widgets.values()) + [continuous_update_checkbox])
                       ])
    tabs.set_title(0,'position')
    tabs.set_title(1,'dimensions')
    tabs.set_title(2,'orientation')
    tabs.set_title(3,'magnetization')
    def rename_magnet(b):
        if b.icon=='check':
            magname_container.children =[rename_magnet_button]
        else:
            magname_container.children =[magname_widget, ok_button]
    
    rename_magnet_button = widgets.Button(description='rename', icon='text', button_style='warning',layout=dict(width='auto'))
    rename_magnet_button.on_click(rename_magnet)
    ok_button = widgets.Button(icon='check', button_style='success',layout=dict(width='auto'))
    ok_button.on_click(rename_magnet)
    magname_container = widgets.HBox([rename_magnet_button])
    
    magnets[mag_id] = {'id': mag_id, 'name':magname_widget, 'trace':figmag.data[-1], 'properties':mag_props, 'magpy_magnet': magpy_magnet,
                                 'widget': widgets.VBox([tabs, widgets.HBox([magname_container ,delete_magnet_button], layout=dict(justify_content='space-between'))])}
    magnets_list_widget.children+=(magnets[mag_id]['widget'],)
    magnets_list_widget.set_title(len(magnets_list_widget.children)-1, f"{mag_id}" if name==None else f"{mag_id} {name}")
    magnets_list_widget.selected_index = len(magnets_list_widget.children)-1
    

# Definitions

In [ ]:
pmc = magpy.Collection()

figmag = go.FigureWidget()
figmag.add_scatter3d(name='ref CS', mode='lines+text', x=[0,10,0,0,0,0], y=[0,0,0,10,0,0], z=[0,0,0,0,0,10], text=['','x','','y','','z'], textfont_color = 'blue')
for k in ['xy', 'xz', 'yz']:
    figmag.add_scatter3d(mode='lines', name = f'streamline {k}')
scene_range = 100
figmag.layout.scene = dict(xaxis_range=[-scene_range,scene_range], yaxis_range=[-scene_range,scene_range], zaxis_range=[-scene_range,scene_range],
                          xaxis_title = 'x[mm]', yaxis_title = 'y[mm]', zaxis_title = 'z[mm]')
figmag.layout.scene.aspectmode = 'cube'
figmag.layout.margin=dict(l=0,r=0,t=10,b=10)

magnets={}


add_magnet_button = widgets.Button(description='add cuboid', icon='plus', button_style='success',layout=dict(width='auto'))
add_magnet_button.on_click(lambda b: add_magnet())

continuous_update_checkbox = widgets.Checkbox(description = 'continuous update', value=True)
continuous_update_checkbox.observe(on_continuous_update_change, names='value')


                                                                             
streamlines_options={}
for k,p in zip(['xy', 'xz', 'yz'],['z', 'y', 'x']):
    sk = streamlines_options[k] = dict()
    sk['checkbox'] = widgets.Checkbox(description=f'{k}-plane', layout=dict(width='auto'), style=dict(description_width='0px'))
    sk['position'] = widgets.FloatSlider(description=f'{p}-position', min=-scene_range, max=scene_range, value=0, step=0.1, 
                                         continuous_update=False, layout=dict(flex='1'))
    sk['density'] = widgets.BoundedFloatText(description='density', min=1, max=5, step = 0.5, value=1, 
                                             continuous_update=False, layout=dict(width='auto'))
    sk['container']= widgets.HBox([sk['checkbox'], sk['position'], sk['density']], layout=dict(justify_content='space-between'))
    
    for s in sk.values():
        s.tag = k
        s.observe(show_streamlines, names='value')
        
streamlines_container = widgets.Accordion([widgets.VBox([v['container'] for v in streamlines_options.values()])])
streamlines_container.set_title(0,'Streamlines')
def set_opacity(opacity):
    for m in magnets.values():
        m['trace'].opacity = opacity
opacity_slider =  widgets.FloatSlider(min=0, max=1, step=0.1, value=1, style=dict(description_width='auto'))
widgets.interactive(set_opacity, opacity =opacity_slider)
magnets_list_widget = widgets.Accordion([])
magnets_window = widgets.Accordion([widgets.VBox([magnets_list_widget, add_magnet_button])])
magnets_window.set_title(0,'Magnets')
graphics_container = widgets.Accordion([widgets.VBox([figmag, opacity_slider])])
graphics_container.set_title(0, 'Graphics')
graphics_container.layout.flex='3'
graphics_container.children[0].layout.max_width='95%'
magnets_window.layout.flex='1 0'
gui = widgets.VBox([widgets.HBox([magnets_window, widgets.VBox([graphics_container, streamlines_container])], layout=dict(flex_flow='wrap')),debug_view])

# Testing

In [ ]:
gui

In [ ]:
#add_magnet(mag=(0,0,1),dim=(80,80,40), pos=(50,50,50), name='testname')
delete_all_magnets()

MX = MY = 80  ;  MZ = 40 # [mm] magnet dimensions
ZB = 22 # [mm] vertical spacing between magnets
XB = 22/np.sqrt(3) # [mm] horizontal spacing between magnets

AX = (XB + MX)/2 # [mm] horizontal distance from the coordinate center to the magnet center
AZ = (ZB + MZ)/2 # [mm] vertical distance from the coordinate center to the magnet center

def add_magnets_example(b):
    add_magnet(mag=(0,0,-1),dim=(80,80,40), pos=(-AX,0,-AZ), name='bottom left')
    add_magnet(mag=(0,0,1),dim=(80,80,40), pos=(+AX,0,-AZ), name='bottom right')
    add_magnet(mag=(0,0,-1),dim=(80,80,40), pos=(-AX,0,+AZ), name='top left')
    add_magnet(mag=(0,0,1),dim=(80,80,40), pos=(+AX,0,+AZ), name='top right')
examples_button = widgets.Button(description='show magnet-set example', layout=dict(width='auto'), button_style='info')
examples_button.on_click(add_magnets_example)
examples_button